In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from torchvision.datasets import ImageFolder
import torchvision.transforms as transform
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
torch.utils.data.Dataset
import torchvision
import pathlib
from torch.optim import Adam
import glob
from torch.autograd import Variable

ImportError: cannot import name '_disabled_torch_function_impl' from 'torch._C' (unknown location)

In [2]:
torch.cuda.is_available() 

True

In [3]:
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650 Ti'

In [4]:
train_path='C:/Users/paltu/anaconda jp/eye/train'
pred_path='C:/Users/paltu/anaconda jp/eye/pred'

In [5]:
import os
from os import listdir

In [6]:
# get the path or directory
# folder_dir = "C:/Users/RIJUSHREE/Desktop/Gfg images"
for images in os.listdir(pred_path)[0:5]:
 
    # check if the image ends with png or jpg or jpeg
    if (images.endswith(".png") or images.endswith(".jpg")\
        or images.endswith(".jpeg")):
        # display
        print(images)

10404_left.jpeg
10404_right.jpeg
10407_left.jpeg
10407_right.jpeg
10408_left.jpeg


In [7]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [8]:
classes

['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']

In [9]:
name_dict={}

name_dict['cataract']=0
name_dict['diabetic_retinopathy']=1
name_dict['glaucoma']=2
name_dict['normal']=3

In [10]:
name_dict

{'cataract': 0, 'diabetic_retinopathy': 1, 'glaucoma': 2, 'normal': 3}

In [11]:
import json
with open("name_dict.json","w") as f:
    f.write(json.dumps(name_dict))

In [12]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=4):
        super(ConvNet,self).__init__()
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.bn2=nn.BatchNorm2d(num_features=20)
        self.relu2=nn.ReLU()
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        
        self.fc=nn.Linear(in_features=75*75*32,out_features=num_classes)
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv2(output)
        output=self.bn2(output)
        output=self.relu2(output)
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        
        
        output=output.view(-1,32*75*75)
        output=self.fc(output)
        
        return output

In [13]:
checkpoint=torch.load('best_checkpoint4.model')
model=ConvNet(num_classes=4)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=4, bias=True)
)

In [14]:
from torchvision.transforms import transforms

In [15]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [16]:
from io import open
import os
from PIL import Image
import pathlib
import glob

In [17]:
def prediction(img_path,transformer):
    image=Image.open(img_path)
    image_tensor=transformer(image).float();
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
    input=Variable(image_tensor)
    
    output=model(input)
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [18]:
images_path1=glob.glob(pred_path+'/*.jpg')

In [19]:
images_path2=glob.glob(pred_path+'/*.jpeg')

In [20]:
images_path3=glob.glob(pred_path+'/*.png')

In [21]:
pred_dict2={}

for i in images_path2:
    pred_dict2[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [22]:
pred_dict2

{'pred\\10404_left.jpeg': 'diabetic_retinopathy',
 'pred\\10404_right.jpeg': 'diabetic_retinopathy',
 'pred\\10407_left.jpeg': 'diabetic_retinopathy',
 'pred\\10407_right.jpeg': 'diabetic_retinopathy',
 'pred\\10408_left.jpeg': 'diabetic_retinopathy',
 'pred\\10408_right.jpeg': 'diabetic_retinopathy',
 'pred\\10409_left.jpeg': 'diabetic_retinopathy',
 'pred\\10409_right.jpeg': 'diabetic_retinopathy',
 'pred\\10752_right.jpeg': 'diabetic_retinopathy',
 'pred\\10755_left.jpeg': 'diabetic_retinopathy',
 'pred\\10755_right.jpeg': 'diabetic_retinopathy',
 'pred\\10758_left.jpeg': 'diabetic_retinopathy',
 'pred\\10758_right.jpeg': 'diabetic_retinopathy',
 'pred\\10761_left.jpeg': 'diabetic_retinopathy',
 'pred\\10761_right.jpeg': 'diabetic_retinopathy',
 'pred\\10762_left.jpeg': 'diabetic_retinopathy',
 'pred\\10762_right.jpeg': 'diabetic_retinopathy',
 'pred\\10763_left.jpeg': 'normal',
 'pred\\10763_right.jpeg': 'diabetic_retinopathy',
 'pred\\1076_left.jpeg': 'diabetic_retinopathy',
 'pre

In [23]:
pred_dict3={}

for i in images_path3:
    pred_dict3[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [24]:
pred_dict3

{'pred\\cataract_039.png': 'glaucoma',
 'pred\\cataract_040.png': 'cataract',
 'pred\\cataract_041.png': 'cataract',
 'pred\\cataract_043.png': 'cataract',
 'pred\\cataract_044.png': 'glaucoma',
 'pred\\cataract_045.png': 'cataract'}

In [25]:
pred_dict={}

for i in images_path1:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [26]:
pred_dict

{'pred\\2244_right.jpg': 'cataract',
 'pred\\2246_left.jpg': 'cataract',
 'pred\\2246_right.jpg': 'cataract',
 'pred\\2247_left.jpg': 'cataract',
 'pred\\2247_right.jpg': 'cataract',
 'pred\\2248_left.jpg': 'cataract',
 'pred\\2248_right.jpg': 'cataract',
 'pred\\2251_right.jpg': 'cataract',
 'pred\\2384_right.jpg': 'normal',
 'pred\\2386_left.jpg': 'normal',
 'pred\\2386_right.jpg': 'normal',
 'pred\\2388_left.jpg': 'normal',
 'pred\\2388_right.jpg': 'normal',
 'pred\\2389_left.jpg': 'normal',
 'pred\\2389_right.jpg': 'normal',
 'pred\\2393_left.jpg': 'normal',
 'pred\\2393_right.jpg': 'normal',
 'pred\\2394_left.jpg': 'normal',
 'pred\\2394_right.jpg': 'normal',
 'pred\\2395_left.jpg': 'normal',
 'pred\\2659_left.jpg': 'normal',
 'pred\\2659_right.jpg': 'normal',
 'pred\\2660_left.jpg': 'normal',
 'pred\\2660_right.jpg': 'normal',
 'pred\\2661_left.jpg': 'normal',
 'pred\\2661_right.jpg': 'normal',
 'pred\\2662_left.jpg': 'normal',
 'pred\\2662_right.jpg': 'normal',
 'pred\\_0_451744

In [27]:
len(images_path1)+len(images_path2)+len(images_path3)

88

In [28]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/'):]]=prediction(i,transformer)

NameError: name 'images_path' is not defined

In [29]:
pred_dict

{}

In [3]:

!pip install tensorflow-addons
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .

     -------------------------------------- 736.6/736.6 kB 2.9 MB/s eta 0:00:00
Obtaining file:///C:/Users/paltu/anaconda%20jp/onnx-tensorflow
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 6.9/6.9 MB 50.7 kB/s eta 0:00:00
     ------------------------------------- 101.3/101.3 kB 38.3 kB/s eta 0:00:00
  Running setup.py develop for onnx-tf


Cloning into 'onnx-tensorflow'...


In [2]:
# load the saved pytorch model and export it as an onnx model

In [1]:
trained_model=Net()
trained_model.load_state_dict()

NameError: name 'Net' is not defined

In [46]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms

# Define the class labels for your model
classes = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']

# Function to preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((150,150))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    image = transform(image).unsqueeze(0)
    return image

# Function to predict the disease
def predict_disease(image_path):
    # Load your pre-trained model here
    # Replace 'model.pth' with the path to your trained model file
    model = ConvNet()  # Replace YourModel with your own model class
    model.load_state_dict(torch.load('best_checkpoint4.model'))
    model.eval()
    
    # Preprocess the image
    image = preprocess_image(image_path)
    
    # Make the prediction
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    
    # Get the predicted class
    predicted_class = classes[predicted.item()]
    return predicted_class

# Function to handle the image selection
def select_image():
    # Open a file dialog to choose an image file
    file_path = filedialog.askopenfilename()
    
    # Display the selected image in the GUI
    image = Image.open(file_path)
    image = image.resize((300, 300))
    image = ImageTk.PhotoImage(image)
    image_label.configure(image=image)
    image_label.image = image
    
    # Call the predict_disease function and display the predicted class
    predicted_class = predict_disease(file_path)
    result_label.config(text="Predicted Disease: " + predicted_class)

# Create the main Tkinter window
window = tk.Tk()
window.title("Disease Prediction")
window.geometry("400x400")

# Create a button to select an image
select_button = tk.Button(window, text="Select Image", command=select_image)
select_button.pack(pady=10)

# Create a label to display the selected image
image_label = tk.Label(window)
image_label.pack()

# Create a label to display the predicted disease
result_label = tk.Label(window, text="Predicted Disease: ")
result_label.pack(pady=10)

# Start the Tkinter event loop
window.mainloop()


In [11]:
!pip install pyinstaller




In [10]:
pip install --upgrade xarray


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\paltu\\anaconda3\\lib\\site-packages\\pandas-1.5.3.dist-info\\METADATA'




  Using cached xarray-2023.5.0-py3-none-any.whl (994 kB)


In [8]:
!pip uninstall pandas


Found existing installation: pandas 1.5.3


ERROR: Cannot uninstall pandas 1.5.3, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps pandas==1.5.3'.


In [9]:
!pip install pandas


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\paltu\\anaconda3\\lib\\site-packages\\pandas-1.5.3.dist-info\\METADATA'



In [41]:
pyinstaller eyeproject2.ipynb

SyntaxError: invalid syntax (3734642276.py, line 1)